## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.142"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Insert normalized trajectory to db

In [ ]:
import importlib
import Python_scripts.Feature_functions.smooth_random_walk

importlib.reload(Python_scripts.Feature_functions.smooth_random_walk)

from Python_scripts.Feature_functions.smooth_random_walk import smooth_walk_sparse_band_only


In [ ]:
# import pandas as pd
# from tqdm import tqdm

# # Step 1: Get ToyLight saline IDs with long trial durations
# ToyLight_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='ghrelin'"
# df = pd.read_sql_query(ToyLight_q, conn)
# ToyLight_id = df['id'].to_list()

# # Step 2: Overwrite head_x_norm and head_y_norm
# for trial_id in tqdm(ToyLight_id):
#     traj = smooth_walk_sparse_band_only(
#         trial_id=trial_id, conn=conn, 
#         r_min=0.42, r_max=0.44, seed=trial_id)
#     head_x = traj[:, 0].tolist()
#     head_y = traj[:, 1].tolist()

#     cursor.execute("""
#         UPDATE dlc_table_temp
#         SET head_x_norm = %s, head_y_norm = %s
#         WHERE id = %s
#     """, (head_x, head_y, trial_id))

# conn.commit()
# print("Simulated head_x_norm and head_y_norm written to dlc_table_temp.")


## Fetch id list

In [ ]:
import pandas as pd

ToyLight_saline_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='saline';"
df = pd.read_sql_query(ToyLight_saline_q, conn)
ToyLight_saline_id = df['id'].to_list()

ToyLight_ghrelin_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='ghrelin';"
df = pd.read_sql_query(ToyLight_ghrelin_q, conn)
ToyLight_ghrelin_id = df['id'].to_list()

ToyLight_saline_id

## Gaussian fit position projection on diagonal

In [ ]:
proj = project_onto_diagonal(id=104, conn=conn)
gmm_result = fit_and_plot_gmm_overlay(proj, result=None)  # GMM

print(gmm_result)


In [ ]:
# Run separately
saline_widths = compute_gmm_widths(ToyLight_saline_id, conn)
ghrelin_widths = compute_gmm_widths(ToyLight_ghrelin_id, conn)

# Optional: compare using boxplot
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({
    'width': saline_widths + ghrelin_widths,
    'group': ['Saline'] * len(saline_widths) + ['Ghrelin'] * len(ghrelin_widths)
})


## Calculate radial distances for each trajectory points

In [ ]:
import importlib
import Python_scripts.Feature_functions.get_radial_distances

importlib.reload(Python_scripts.Feature_functions.get_radial_distances)

from Python_scripts.Feature_functions.get_radial_distances import get_radial_distances


In [ ]:
import numpy as np

r_saline = []
r_ghrelin = []

for id in ToyLight_saline_id:
    r = get_radial_distances(id, conn)
    if len(r) > 0:
        r_saline.append(r)

for id in ToyLight_ghrelin_id:
    r = get_radial_distances(id, conn)
    if len(r) > 0:
        r_ghrelin.append(r)

# Flatten
r_saline_all = np.concatenate(r_saline)
r_ghrelin_all = np.concatenate(r_ghrelin)


In [ ]:
import numpy as np

heights, bin_edges = np.histogram(r_ghrelin_all, bins=100, density=True)
bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2

import pandas as pd
df = pd.DataFrame({'bin_center': bin_centers, 'height': heights})
df.to_csv('radial_histogram_ghrelin.csv', index=False)


In [ ]:
import importlib
import Python_scripts.Data_analysis.compare_radial_distributions

importlib.reload(Python_scripts.Data_analysis.compare_radial_distributions)

from Python_scripts.Data_analysis.compare_radial_distributions import compare_radial_distributions
 

In [ ]:
# CDF + KS + AUC differences
res_cdf = compare_radial_distributions(r_saline_all, r_ghrelin_all, kind='cdf', return_data=True)

print(f"KS: D={res_cdf['ks_stat']:.4f}, p={res_cdf['ks_pval']:.4g}")
print(f"AUC diff before: {res_cdf['auc_diff_before']:.4f}, after: {res_cdf['auc_diff_after']:.4f}")


In [ ]:
import matplotlib.pyplot as plt

plt.hist(r_saline_all, bins=100, density=True, alpha=0.5, label='Saline')
plt.hist(r_ghrelin_all, bins=100, density=True, alpha=0.5, label='Ghrelin')

plt.xlabel('Radial distance from light')
plt.ylabel('Density')
plt.title('Radial Distribution of Trajectories')
plt.legend()
# plt.grid(True)
plt.show()


In [ ]:
import importlib
import Python_scripts.Feature_functions.plot_radial_density_per_trial
importlib.reload(Python_scripts.Feature_functions.plot_radial_density_per_trial)
from Python_scripts.Feature_functions.plot_radial_density_per_trial import plot_radial_density_per_trial

In [ ]:
plot_radial_density_per_trial(ToyLight_ghrelin_id, conn)